In [7]:
import pyspark
#sc = pyspark.SparkContext('local[*]')
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
# needed whenever working with spark dataframes
from pyspark.sql import *
!rm -rf metastore_db/
sqlContext = SQLContext(sc)

In [3]:
!wget https://ibm.box.com/shared/static/f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv

--2018-10-06 14:40:34--  https://ibm.box.com/shared/static/f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv [following]
--2018-10-06 14:40:34--  https://ibm.ent.box.com/shared/static/f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv
Resolving ibm.ent.box.com (ibm.ent.box.com)... 185.235.236.211
Connecting to ibm.ent.box.com (ibm.ent.box.com)|185.235.236.211|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcloud.com/d/1/b1!I7xfBxzQrxUIihY4VuOxDoHkB_KsLWVylRjuANAIAORfcmrJdg-osOoMeL6NB8A0ucPt6UNpEhex3s2S86Uwycwpf1bOBpvTAycLZBj-rpBSPFW_Rdw9kxeavaqiiNMx8nbLe2lcV2Wsy4YLLpXa5gcQFHIVkgwbNYaEVTLcR_zgN1JzCOlhkz33P-OL8ajaWYtzwJKcOXf-cI1Zl3-0OyCv8vvOCmjQthf1E0GRUM5I14tjjLLbG6im8rQvYnJavLiAJSSyTm4iSO3E3i

In [41]:
# using inferschema is very handy, but adds a second pass over the data
df = sqlContext.read.format('com.databricks.spark.csv')\
                .option('header', 'true')\
                .option('inferschema', 'true')\
                .option('mode', 'DROPMALFORMED')\
                .load('f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv')
#look at Spark Web UI to see the two passes

In [18]:
df.filter('gear == 4').take(5)

[Row(car='Mazda RX4', mpg=21.0, cyl=6, disp=160.0, hp=110, drat=3.9, wt=2.62, qsec=16.46, vs=0, am=1, gear=4, carb=4),
 Row(car='Mazda RX4 Wag', mpg=21.0, cyl=6, disp=160.0, hp=110, drat=3.9, wt=2.875, qsec=17.02, vs=0, am=1, gear=4, carb=4),
 Row(car='Datsun 710', mpg=22.8, cyl=4, disp=108.0, hp=93, drat=3.85, wt=2.32, qsec=18.61, vs=1, am=1, gear=4, carb=1),
 Row(car='Merc 240D', mpg=24.4, cyl=4, disp=146.7, hp=62, drat=3.69, wt=3.19, qsec=20.0, vs=1, am=0, gear=4, carb=2),
 Row(car='Merc 230', mpg=22.8, cyl=4, disp=140.8, hp=95, drat=3.92, wt=3.15, qsec=22.9, vs=1, am=0, gear=4, carb=2)]

In [21]:
df.count()

32

In [22]:
df.printSchema()

root
 |-- car: string (nullable = true)
 |-- mpg: double (nullable = true)
 |-- cyl: integer (nullable = true)
 |-- disp: double (nullable = true)
 |-- hp: integer (nullable = true)
 |-- drat: double (nullable = true)
 |-- wt: double (nullable = true)
 |-- qsec: double (nullable = true)
 |-- vs: integer (nullable = true)
 |-- am: integer (nullable = true)
 |-- gear: integer (nullable = true)
 |-- carb: integer (nullable = true)



In [23]:
df.show(5)

+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|              car| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|        Mazda RX4|21.0|  6|160.0|110| 3.9| 2.62|16.46|  0|  1|   4|   4|
|    Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|
|       Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|   Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|
|Hornet Sportabout|18.7|  8|360.0|175|3.15| 3.44|17.02|  0|  0|   3|   2|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
only showing top 5 rows



In [24]:
df.select('car', 'mpg').show(5)

+-----------------+----+
|              car| mpg|
+-----------------+----+
|        Mazda RX4|21.0|
|    Mazda RX4 Wag|21.0|
|       Datsun 710|22.8|
|   Hornet 4 Drive|21.4|
|Hornet Sportabout|18.7|
+-----------------+----+
only showing top 5 rows



In [25]:
df.filter(df['mpg'] < 18).show(5)

+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
|        car| mpg|cyl| disp| hp|drat|  wt| qsec| vs| am|gear|carb|
+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
| Duster 360|14.3|  8|360.0|245|3.21|3.57|15.84|  0|  0|   3|   4|
|  Merc 280C|17.8|  6|167.6|123|3.92|3.44| 18.9|  1|  0|   4|   4|
| Merc 450SE|16.4|  8|275.8|180|3.07|4.07| 17.4|  0|  0|   3|   3|
| Merc 450SL|17.3|  8|275.8|180|3.07|3.73| 17.6|  0|  0|   3|   3|
|Merc 450SLC|15.2|  8|275.8|180|3.07|3.78| 18.0|  0|  0|   3|   3|
+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
only showing top 5 rows



In [34]:
summary = df.describe().toPandas().set_index('summary')
summary

,car,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
summary,,,,,,,,,,,,
count,32,32,32,32,32,32,32,32,32,32,32,32
mean,None,20.090624999999996,6.1875,230.72187500000004,146.6875,3.5965625000000006,3.2172499999999995,17.848750000000003,0.4375,0.40625,3.6875,2.8125
stddev,None,6.026948052089103,1.7859216469465444,123.93869383138195,68.56286848932059,0.5346787360709716,0.9784574429896968,1.7869432360968436,0.5040161287741853,0.49899091723584604,0.7378040652569471,1.6151999776318522
min,AMC Javelin,10.4,4,71.1,52,2.76,1.513,14.5,0,0,3,1
max,Volvo 142E,33.9,8,472.0,335,4.93,5.424,22.9,1,1,5,8


In [35]:
summary.loc['mean', 'gear']

'3.6875'

In [10]:
# the filter can be just like a SQL where clause
df.filter('mpg < 18').show(5)

+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
|        car| mpg|cyl| disp| hp|drat|  wt| qsec| vs| am|gear|carb|
+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
| Duster 360|14.3|  8|360.0|245|3.21|3.57|15.84|  0|  0|   3|   4|
|  Merc 280C|17.8|  6|167.6|123|3.92|3.44| 18.9|  1|  0|   4|   4|
| Merc 450SE|16.4|  8|275.8|180|3.07|4.07| 17.4|  0|  0|   3|   3|
| Merc 450SL|17.3|  8|275.8|180|3.07|3.73| 17.6|  0|  0|   3|   3|
|Merc 450SLC|15.2|  8|275.8|180|3.07|3.78| 18.0|  0|  0|   3|   3|
+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
only showing top 5 rows



In [42]:
# dataframes are immutable -- withColumn creates a new one
df = df.withColumn('wtTon', df['wt'] * 0.45)

In [43]:
df.show(1)

+---------+----+---+-----+---+----+----+-----+---+---+----+----+-----+
|      car| mpg|cyl| disp| hp|drat|  wt| qsec| vs| am|gear|carb|wtTon|
+---------+----+---+-----+---+----+----+-----+---+---+----+----+-----+
|Mazda RX4|21.0|  6|160.0|110| 3.9|2.62|16.46|  0|  1|   4|   4|1.179|
+---------+----+---+-----+---+----+----+-----+---+---+----+----+-----+
only showing top 1 row



In [49]:
from pyspark.sql import functions as F

In [54]:
F.mean('wt').alias('mean_wt')

Column<b'avg(wt) AS `mean_wt`'>

In [52]:
df.groupby(['cyl']).agg(F.mean('wt').alias('mean_wt'), F.stddev('wt'), F.sum('wt')).show()

+---+------------------+-------------------+-----------------+
|cyl|           mean_wt|    stddev_samp(wt)|          sum(wt)|
+---+------------------+-------------------+-----------------+
|  6| 3.117142857142857|0.35634553856775425|            21.82|
|  4| 2.285727272727273| 0.5695637086242575|           25.143|
|  8|3.9992142857142867| 0.7594047444769265|55.98900000000001|
+---+------------------+-------------------+-----------------+



In [55]:
df.groupby(['cyl'])\
.agg({"wt": "AVG"})\
.show(5)

+---+------------------+
|cyl|           avg(wt)|
+---+------------------+
|  6| 3.117142857142857|
|  4| 2.285727272727273|
|  8|3.9992142857142867|
+---+------------------+



In [58]:
g = df.groupby(['cyl'])
g.mean().show()

+---+------------------+--------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+
|cyl|          avg(mpg)|avg(cyl)|         avg(disp)|           avg(hp)|         avg(drat)|           avg(wt)|         avg(qsec)|           avg(vs)|            avg(am)|         avg(gear)|         avg(carb)|        avg(wtTon)|
+---+------------------+--------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+
|  6| 19.74285714285714|     6.0|183.31428571428572|122.28571428571429|3.5857142857142854| 3.117142857142857| 17.97714285714286|0.5714285714285714|0.42857142857142855| 3.857142857142857|3.4285714285714284|1.4027142857142856|
|  4|26.663636363636364|     4.0|105.13636363636364| 82.63636363636364| 4.070909090909091| 2.2857272

In [13]:
car_counts = df.groupby(['cyl'])\
.agg({"*": "count"})\
.sort("count(1)", ascending=False)\
.show(5)

+---+--------+
|cyl|count(1)|
+---+--------+
|  8|      14|
|  4|      11|
|  6|       7|
+---+--------+



In [61]:
df.createOrReplaceTempView("mytable")

# SQL statements can be run by using the sql method
highgearcars = sqlContext.sql("SELECT gear FROM mytable WHERE cyl >= 4 AND cyl <= 9")
highgearcars.show(6)

+----+
|gear|
+----+
|   4|
|   4|
|   4|
|   3|
|   3|
|   3|
+----+
only showing top 6 rows



## Exercise 1:

Step 1: Create a list of tuples ("john", 23), ("mr. bean", 56), ("bill clinton", 70)

Step 2: Convert the list to a data frame with two columns

Step 3: Filter the people with age < 40

Step 4: Get the names ONLY of people smaller than 40

Step 5: Register the dataframe as a SQL table

Step 6: Select the names only of people whose age < 40

Some hints: http://spark.apache.org/docs/latest/sql-programming-guide.html#generic-loadsave-functions

## Exercise 2:

Step 1: Create a list of tuple ("john", "male"), ("hilary", "female")

Step 2: Join with the data from the previos exercise and output all males older than 50

Hints: https://databricks.com/blog/2015/02/17/introducing-dataframes-in-spark-for-large-scale-data-science.html

In [16]:
people = [("john", 23), ("mr. bean", 56), ("bill clinton", 70)]

In [17]:
df_people = sqlContext.createDataFrame(people, ['name', 'age'])
df_people.show()

+------------+---+
|        name|age|
+------------+---+
|        john| 23|
|    mr. bean| 56|
|bill clinton| 70|
+------------+---+



In [18]:
df_people.where('age < 40').show()

+----+---+
|name|age|
+----+---+
|john| 23|
+----+---+



In [19]:
df_people.select('name').where('age < 40').show()

+----+
|name|
+----+
|john|
+----+



In [20]:
df_people.createOrReplaceTempView('tbl_people')

In [21]:
sqlContext.sql('select name from tbl_people where age < 40').show()

+----+
|name|
+----+
|john|
+----+



In [ ]:
sc.stop()